<a href="https://colab.research.google.com/github/hieu1999210/Insincere-question-classification/blob/master/group_Relu_GoogleNews_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/gdrive')
# mount nguyentrunghieu

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [3]:
cd "/gdrive/My Drive"

/gdrive/My Drive


In [0]:
EMBEDDING_FILE = 'embeddings/GoogleNews-vectors-negative300.bin'

In [5]:
path_train = "nlp/data/train.csv"
path_test = "nlp/data/test.csv"
train = pd.read_csv(path_train)
# test = pd.read_csv(path_test)
print(train.shape)
# print(test.shape)

(1306122, 3)


In [0]:
train, val = train_test_split(train, test_size = 0.1, random_state = 2019) 

In [0]:
embed_size = 300
max_features = 50000
maxlen = 100

In [0]:
## fill up the missing values
train_X = train["question_text"].fillna("_na_").values
val_X = val["question_text"].fillna("_na_").values
# test_X = test["question_text"].fillna("_na_").values

#Tokenize the sentences
tokenizer = Tokenizer(num_words = max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
# test_X = tokenizer.texts_to_sequences(test_X)

In [0]:
#padding
train_X = pad_sequences(train_X, maxlen = maxlen)
val_X = pad_sequences(val_X, maxlen = maxlen)
# test_X = pad_sequences(test_X, maxlen = maxlen)


In [0]:
train_Y = train['target'].values
val_Y = val['target'].values

In [11]:
print(train_Y.shape)
print(val_Y.shape)

(1175509,)
(130613,)


In [12]:
from gensim.models import KeyedVectors

embeddings_index = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((50000, 300))
for word, i in word_index.items():
    if i >= max_features: continue
    if word in embeddings_index:
        embedding_vector = embeddings_index.get_vector(word)
        embedding_matrix[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [13]:
!pip install h5py pyyaml
# !pip install tf_nightly

In [14]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.14.0'

In [0]:
import tensorflow as tf
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(y_true * y_pred)
        possible_positives = K.sum(y_true)
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(y_true * y_pred)
        predicted_positives = K.sum(y_pred)
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    Y_pred = tf.cast(tf.less(0.4, y_pred), tf.float32)
    
    precision = precision(y_true, Y_pred)
    recall = recall(y_true, Y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

def create_model():
    model = Sequential()
    model.add(Embedding(50000, 300, input_length=100, weights=[embedding_matrix], trainable=False))
    model.add(Dropout(0.2))
    model.add(Conv1D(64, 5, activation='relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(LSTM(300))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])
    
    return model
model = create_model()

model.summary()


W0706 16:18:03.079189 139782934517632 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0706 16:18:03.141601 139782934517632 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0706 16:18:03.159790 139782934517632 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0706 16:18:03.179812 139782934517632 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0706 16:18:03.180612 1397829345

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 300)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 96, 64)            96064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 24, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 300)               438000    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 301       
Total params: 15,534,365
Trainable params: 534,365
Non-trainable params: 15,000,000
__________________________________________________________

In [17]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=2)
model.save_weights(checkpoint_path.format(epoch=0))
history = model.fit(train_X, train_Y, batch_size=512, epochs=10, callbacks = [cp_callback],validation_data=(val_X, val_Y))

W0706 16:18:07.069154 139782934517632 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Train on 1175509 samples, validate on 130613 samples
Epoch 1/10
1175509/1175509 [==============================] - 138s 118us/step - loss: 0.1287 - f1: 0.5431 - val_loss: 0.1182 - val_f1: 0.6089
Epoch 2/10
1175509/1175509 [==============================] - 134s 114us/step - loss: 0.1143 - f1: 0.6085 - val_loss: 0.1206 - val_f1: 0.6298

Epoch 00002: saving model to training_2/cp-0002.ckpt
Epoch 3/10
1175509/1175509 [==============================] - 134s 114us/step - loss: 0.1098 - f1: 0.6267 - val_loss: 0.1132 - val_f1: 0.6384
Epoch 4/10
1175509/1175509 [==============================] - 134s 114us/step - loss: 0.1064 - f1: 0.6410 - val_loss: 0.1175 - val_f1: 0.6372

Epoch 00004: saving model to training_2/cp-0004.ckpt
Epoch 5/10
1175509/1175509 [==============================] - 134s 114us/step - loss: 0.1037 - f1: 0.6513 - val_loss: 0.1121 - val_f1: 0.6476
Epoch 6/10
1175509/1175509 [==============================] - 133s 113us/step - loss: 0.1013 - f1: 0.6585 - val_loss: 0.1130 - va

AttributeError: ignored

In [0]:
model.save_weights("Naive_weights_GoogleNews.h5")

In [19]:
predict_val_y = model.predict(val_X, batch_size = 1024, verbose = 1)



130613/130613 [==============================] - 4s 30us/step


In [20]:
history.history.keys()

NameError: ignored

In [0]:
import matplotlib.pyplot as plt

plt.plot(history.history['f1'])
plt.plot(history.history['val_f1'])
plt.title('model performance')
plt.ylabel('f1')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')

plt.show()

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper right')

plt.show()

In [21]:
scores = []
for thresh in np.arange(0.1, 0.901, 0.01):
    thresh = np.round(thresh, 2)
    score = metrics.f1_score(val_Y, (predict_val_y>thresh).astype(int))
    scores.append(score)
    print("F1 score at threshold {0} is {1}".format(thresh, score))
    
print("Best F1 score:", max(scores))

F1 score at threshold 0.1 is 0.5242086648983201
F1 score at threshold 0.11 is 0.5336279019231465
F1 score at threshold 0.12 is 0.542773517098292
F1 score at threshold 0.13 is 0.5491320754716981
F1 score at threshold 0.14 is 0.5566455087516831
F1 score at threshold 0.15 is 0.5626346086071191
F1 score at threshold 0.16 is 0.5702436495593571
F1 score at threshold 0.17 is 0.5760447035957241
F1 score at threshold 0.18 is 0.5808626999465439
F1 score at threshold 0.19 is 0.5867023053792182
F1 score at threshold 0.2 is 0.5908436997545908
F1 score at threshold 0.21 is 0.5947614352467098
F1 score at threshold 0.22 is 0.5989058700937826
F1 score at threshold 0.23 is 0.6028745989187289
F1 score at threshold 0.24 is 0.6066018037229552
F1 score at threshold 0.25 is 0.6101603557472038
F1 score at threshold 0.26 is 0.612827034883721
F1 score at threshold 0.27 is 0.6162112514351321
F1 score at threshold 0.28 is 0.6191979205347197
F1 score at threshold 0.29 is 0.6220014992503747
F1 score at threshold 0.